In [1]:
# sdkvm

In [7]:
import pandas as pd
import joblib
import lightgbm as lgbm
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,classification_report

In [8]:
train_df = pd.read_csv(".././Training-Testing Dataset/Training Data (RWS Oversampling).csv")
test_df = pd.read_csv(".././Training-Testing Dataset/Testing Data CC undersampling.csv")
target_col = "target"

X_train = train_df.drop(columns=[target_col])
X_test = test_df.drop(columns=[target_col])

y_train = train_df[target_col]
y_test  = test_df[target_col]

print(y_train.value_counts())
categorical_columns = [col for col in X_train.columns if X_train[col].dtype == 'object']

for col in categorical_columns:
    X_train[col] = X_train[col].astype("category")
    X_test[col] = X_test[col].astype("category")

#LGBM only accepts numerical data in the target columns
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

train_set = lgbm.Dataset(X_train,label=y_train,categorical_feature=categorical_columns)
test_set = lgbm.Dataset(X_test,label=y_test,categorical_feature=categorical_columns)


target
Medium_Yes    4492
Low_No        4492
Medium_No     4492
High_No       4492
Low_Yes       4492
High_Yes      4492
Name: count, dtype: int64


In [9]:
print(len(set(y_train)))

6


In [10]:
params = {
    'objective':'multiclass',
    'num_class':len(set(y_train)),
    'metric':'multi_error',
    'learning_rate':0.09,
    'num_leaves':70,
    'max_depth':-1,
    'verbose':-1
}

model = lgbm.train(params,train_set,valid_sets=[test_set],num_boost_round=500)

In [11]:
# Predict
y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)  # for multiclass
print("Accuracy:", accuracy_score(y_test, y_pred_classes))
print(classification_report(y_test, y_pred_classes))

Accuracy: 0.716376582278481
              precision    recall  f1-score   support

           0       0.59      0.46      0.52       509
           1       0.88      0.98      0.93       304
           2       0.65      0.57      0.61       509
           3       0.86      1.00      0.93       188
           4       0.52      0.60      0.56       509
           5       0.91      0.97      0.94       509

    accuracy                           0.72      2528
   macro avg       0.74      0.76      0.75      2528
weighted avg       0.71      0.72      0.71      2528



In [12]:
joblib.dump(model,"lightGBM_model.pkl")

['lightGBM_model.pkl']

In [26]:
model = joblib.load("lightGBM_model.pkl")

In [27]:
test_df = pd.read_csv(".././Training-Testing Dataset/Testing Data CC undersampling.csv")

target_col = "target"   

X_test = test_df.drop(columns=[target_col])

y_test  = test_df[target_col]

print(y_test.value_counts())

for col in categorical_columns:
    X_test[col] = X_test[col].astype("category")

y_test = le.fit_transform(y_test)


target
High_No       509
Low_No        509
Medium_Yes    509
Medium_No     509
High_Yes      304
Low_Yes       188
Name: count, dtype: int64


In [28]:
y_pred = model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)  # for multiclass
print("Accuracy:", accuracy_score(y_test, y_pred_classes))
print(classification_report(y_test, y_pred_classes))

Accuracy: 0.716376582278481
              precision    recall  f1-score   support

           0       0.59      0.46      0.52       509
           1       0.88      0.98      0.93       304
           2       0.65      0.57      0.61       509
           3       0.86      1.00      0.93       188
           4       0.52      0.60      0.56       509
           5       0.91      0.97      0.94       509

    accuracy                           0.72      2528
   macro avg       0.74      0.76      0.75      2528
weighted avg       0.71      0.72      0.71      2528

